# The Battle of Neighborhood (Week 1)
### Business Problem

**Background** \
According to the annual Demographia international housing affordability survey, Sydney ranked third and Melbourne fourth compared to 309 housing markets across eight countries including some of the most expensive cities in the world such as Hong Kong or Vancouver. Melbourne's housing market is heating up, as spring is coming. “Spring is a traditional strong sales period, and with fresh demand from buyers, now is the time for anyone thinking about selling a house or a unit to act”, Real Estate Institute of Victoria CEO Gil King says. However, in such a scenario, this situation imposes a big concern for homebuyers: *Among hundred thousands available real estate on sale, which one is both most suitable and affordable for my preferences and financial status?* 

**Business Problem** \
This analysis is for anyone who is planning to buy/rent a new house/apartment in Melbourne. In my opinion, it is necesarry to adopt machine learning and data analysis to provide recommendations for them to decide or purchase a suitable and reasonably priced real estate in Melbourne. \
To solve this business problem, we are going to cluster Melbourne neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues based on amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

### Data Collection

Data on Melbourne properties and price was scraped from publicly available results posted every week from DomainGroup < https://www.domain.com.au/>. The dataset includes almost every information you request of a property, such as Address, Type of Real estate, Suburb, Method of Selling, Rooms, Price, Real Estate Agent, Date of Sale. The dataset even contains location coordinates of properties for the sake of clustering and visualizing. In order to better understand the dataset, here are some notes: 

|    Key    |    Meaning    |  |  |  |
|---------------------|-----------------------------------------------------------------------------|---|---|---|
|    Suburb    |    Suburb    |  |  |  |
|    Rooms    |    Number of rooms    |  |  |  |
|    Price    |    Price in Australian   dollars    |  |  |  |
|    Br    |    Bedroom(s)    |  |  |  |
|    H    |    House, cottage,   villa, semi, terrace    |  |  |  |
|    U    |    Unit, duplex    |  |  |  |
|    T    |    Townhouse    |  |  |  |
|    Dev site    |    Development site    |  |  |  |
|    O res    |    Other residential    |  |  |  |
|    Distance    |    Distance from   Central Business District (City Centre) in kilometres    |  |  |  |
|    Regionname    |    General Region (West,   North West, North, etc.)    |  |  |  |
|    Propertycount    |    Number of properties   that exist in the suburb    |  |  |  |
|    CouncilArea    |    Governing council   for the area    |  |  |  |

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on Melbourne properties and price and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend suitable real estate investments.